In [1]:
import pandas as pd
import datetime
pd.set_option('display.width', 1000)

In [2]:
user_interactions = pd.read_csv('./data/user-interactions.csv')
user_interactions.head()

KeyboardInterrupt: 

In [ ]:
# 무의하다고 판단되는 clolumn 1차 제거
user_interactions_df = user_interactions.drop(columns=['Unnamed: 0', 'user_session', 'target'])
print(user_interactions_df.shape)
user_interactions_df.isnull().sum()

(11495242, 10)


event_time           0
event_type           0
product_id           0
brand           949132
price                0
user_id              0
cat_0          1527738
cat_1          1527738
cat_2          5071243
cat_3         11488757
dtype: int64

In [ ]:
# Null 값이 너무 많은 column 제거
user_interactions_df.drop(columns=['cat_3'], inplace=True)
user_interactions_df = user_interactions_df.dropna(axis=0)
user_interactions_df.shape

(5997796, 9)

In [ ]:
#  Null 값 확인
user_interactions_df.isnull().sum()

event_time    0
event_type    0
product_id    0
brand         0
price         0
user_id       0
cat_0         0
cat_1         0
cat_2         0
dtype: int64

In [ ]:
# Dataframe 요약 정보
user_interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5997796 entries, 4 to 11495241
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_time  object 
 1   event_type  object 
 2   product_id  int64  
 3   brand       object 
 4   price       float64
 5   user_id     int64  
 6   cat_0       object 
 7   cat_1       object 
 8   cat_2       object 
dtypes: float64(1), int64(2), object(6)
memory usage: 457.6+ MB


- event_time:  object --> Unix timestamp 타입 변환 필요

In [ ]:
#  통계량 요약
user_interactions_df.describe()

,product_id,price,user_id
count,5.997796e+06,5.997796e+06,5.997796e+06
mean,1.013193e+07,3.377886e+02,5.552734e+08
std,2.421850e+07,3.461352e+02,3.311072e+07
min,1.000978e+06,7.700000e-01,3.948059e+07
25%,1.005003e+06,1.196900e+02,5.209589e+08
50%,1.005259e+06,2.084700e+02,5.538288e+08
75%,4.803637e+06,4.335900e+02,5.841051e+08
max,1.001444e+08,2.574070e+03,6.220901e+08


- price 값의 변동량이 너무 크므로 MinMax Scaler 필요

### evnet_time type conversion: object to Unix timestamp

In [ ]:
user_interactions_df2 = user_interactions_df.copy()

# 유효하지 않는  format은  NaT으로 변환
user_interactions_df2['event_time'] = pd.to_datetime(user_interactions_df2['event_time'], format="%Y-%m-%d %H:%M:%S UTC", errors='coerce')
user_interactions_df2.dropna()

,event_time,event_type,product_id,brand,price,user_id,cat_0,cat_1,cat_2
4,2022-11-01 00:10:45,cart,4804056,apple,160.57,522355747,electronics,audio,headphone
13,2022-11-01 00:24:48,cart,4804295,xiaomi,22.80,550508533,electronics,audio,headphone
14,2022-11-01 00:25:12,cart,4804295,xiaomi,22.80,550508533,electronics,audio,headphone
17,2022-11-01 00:31:35,cart,1801881,samsung,488.80,566283686,electronics,video,tv
35,2022-11-01 00:49:04,cart,3700689,samsung,191.74,529765040,appliances,environment,vacuum
...,...,...,...,...,...,...,...,...,...
11495235,2023-01-31 23:57:38,purchase,1005184,samsung,977.86,595018267,appliances,kitchen,refrigerators
11495236,2023-01-31 23:57:52,purchase,26300086,lucente,237.84,514851004,appliances,kitchen,coffee_grinder
11495238,2023-01-31 23:58:35,purchase,1004210,samsung,79.54,571761801,construction,tools,light
11495240,2023-01-31 23:59:39,purchase,1005186,samsung,720.46,599883250,appliances,kitchen,refrigerators


In [ ]:

# event_time의 datetime을 Unix timestamp 타입으로
user_interactions_df2['event_time'] = user_interactions_df2.event_time.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
user_interactions_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5997796 entries, 4 to 11495241
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_time  float64
 1   event_type  object 
 2   product_id  int64  
 3   brand       object 
 4   price       float64
 5   user_id     int64  
 6   cat_0       object 
 7   cat_1       object 
 8   cat_2       object 
dtypes: float64(2), int64(2), object(5)
memory usage: 586.6+ MB


In [ ]:
# price MinMax Sacler 적용
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
user_interactions_df2['price'] = scaler.fit_transform(user_interactions_df2[['price']])

In [ ]:
user_interactions_df2.head()

,event_time,event_type,product_id,brand,price,user_id,cat_0,cat_1,cat_2
4,1.667261e+09,cart,4804056,apple,0.062099,522355747,electronics,audio,headphone
13,1.667262e+09,cart,4804295,xiaomi,0.008561,550508533,electronics,audio,headphone
14,1.667262e+09,cart,4804295,xiaomi,0.008561,550508533,electronics,audio,headphone
17,1.667263e+09,cart,1801881,samsung,0.189651,566283686,electronics,video,tv
35,1.667264e+09,cart,3700689,samsung,0.074212,529765040,appliances,environment,vacuum


In [ ]:
# rename columne:  Personalize를 위한 필수 항목: USER_ID, ITEM_ID, TIMESTAMP
user_interactions_df2.rename(columns={"user_id": "USER_ID", "product_id": "ITEM_ID", "event_time": "TIMESTAMP", "event_type": "EVENT_TYPE", "brand": "BRAND",
                                      "price": "PRICE", "cat_0": "CAT_0", "cat_1": "CAT_1", "cat_2": "CAT_2" }, inplace=True)
user_interactions_df2 = user_interactions_df2.reindex(columns=['USER_ID', 'ITEM_ID', 'TIMESTAMP', "EVENT_TYPE", "PRICE", "BRAND", "CAT_0", "CAT_1", "CAT_2"])

In [ ]:
user_interactions_df2.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE,PRICE,BRAND,CAT_0,CAT_1,CAT_2
4,522355747,4804056,1.667261e+09,cart,0.062099,apple,electronics,audio,headphone
13,550508533,4804295,1.667262e+09,cart,0.008561,xiaomi,electronics,audio,headphone
14,550508533,4804295,1.667262e+09,cart,0.008561,xiaomi,electronics,audio,headphone
17,566283686,1801881,1.667263e+09,cart,0.189651,samsung,electronics,video,tv
35,529765040,3700689,1.667264e+09,cart,0.074212,samsung,appliances,environment,vacuum


In [ ]:
# 정제된 데이터를 파일로 저장
user_interactions_df2.to_csv("./data/cleaned_user_interactions.csv", index=False)

In [3]:
interactions = pd.read_csv('./data/cleaned_user_interactions.csv')
print(interactions.shape)
interactions.head()

(5997796, 9)


,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE,PRICE,BRAND,CAT_0,CAT_1,CAT_2
0,522355747,4804056,1.667261e+09,cart,0.062099,apple,electronics,audio,headphone
1,550508533,4804295,1.667262e+09,cart,0.008561,xiaomi,electronics,audio,headphone
2,550508533,4804295,1.667262e+09,cart,0.008561,xiaomi,electronics,audio,headphone
3,566283686,1801881,1.667263e+09,cart,0.189651,samsung,electronics,video,tv
4,529765040,3700689,1.667264e+09,cart,0.074212,samsung,appliances,environment,vacuum


In [15]:
# At minimum 1000 interactions records from users interacting with items in your catalog.
# At minimum 25 unique user IDs with at least 2 interactions for each.

interactions['USER_ID'].value_counts() >= 3

568782581     True
513230794     True
549109608     True
553431815     True
582826305     True
             ...  
556264149    False
520949146    False
586824173    False
550128253    False
514851004    False
Name: USER_ID, Length: 1607025, dtype: bool